In [4]:
import pandas as pd
import glob
import xarray as xr
import numpy as np
from statsmodels.stats.anova import AnovaRM 
from scipy import stats
from matplotlib import pyplot as plt
import pingouin as pg
import numpy as np

In [12]:
vn = ['fusiform-rh', 'fusiform-lh', 'lingual-rh', 'lingual-lh', 'cuneus-rh', 'cuneus-lh', 'lateraloccipital-rh', 'lateraloccipital-lh']
methods = ['coh', 'ciplv','imcoh','wpli2']
conditions = ['EO','EC']
xarrays = glob.glob(f'/Users/lina_01/Desktop/mpi_lemon/undirected_outputs/coh/*_coh_EC.nc')
subjects = []
for array in xarrays:
    subject = array.split('/')[7].split('_')[0]
    subjects.append(subject)

In [ ]:
#CALCULATE INTER-SUBJECT VARIANCE

In [13]:
inter_subject_variance = {}
for condition in conditions:
    condition_dict = {}
    for method in methods:
        mean_per_subject = []
        for subject in subjects:
            xarray = xr.open_dataarray(f'/Users/lina_01/Desktop/mpi_lemon/undirected_outputs/{method}/{subject}_array_{method}_{condition}.nc')
            xarray_sel = xarray.sel(region1=vn, region2=vn)
            mean = float(xarray_sel.mean())
            mean_per_subject.append(mean)
            
        variance = np.std(mean_per_subject)**2
        condition_dict[method] = variance
    inter_subject_variance[condition] = condition_dict

In [ ]:
#CALCULATE INTRA-SUBJECT VARIANCE

In [14]:
intra_subject_variance = {}
for condition in conditions:
    condition_dict = {}
    for method in methods:
        variance_per_subject = []
        for subject in subjects:
            xarray = xr.open_dataarray(f'/Users/lina_01/Desktop/mpi_lemon/undirected_outputs/{method}/{subject}_array_{method}_{condition}.nc')
            mean_per_sample = []
            for sample in range(0,100):
                xarray_sel = xarray.sel(bootstrap_samples=sample, region1=vn, region2=vn)
                mean = float(xarray_sel.mean())
                mean_per_sample.append(mean)
    
            variance = np.std(mean_per_sample)**2
            variance_per_subject.append(variance)
        condition_dict[method] = np.mean(variance_per_subject)
    intra_subject_variance[condition] = condition_dict
                

In [16]:
inter_subject_variance

{'EO': {'coh': 0.0007089194926995154,
  'ciplv': 1.589471735562101e-05,
  'imcoh': 3.933224046587579e-06,
  'wpli2': 1.2916318221102206e-05},
 'EC': {'coh': 0.00045111117928560814,
  'ciplv': 8.110093226079214e-05,
  'imcoh': 9.315652210398905e-06,
  'wpli2': 8.621990031468762e-05}}

In [15]:
intra_subject_variance

{'EO': {'coh': 4.132560584572334e-06,
  'ciplv': 5.269786543493939e-06,
  'imcoh': 1.6921321643831103e-06,
  'wpli2': 2.263572002978859e-06},
 'EC': {'coh': 3.540251858764824e-06,
  'ciplv': 5.8168599400247114e-06,
  'imcoh': 1.4503351962160827e-06,
  'wpli2': 3.7691857911563666e-06}}

In [ ]:
#CALCULATE RATIOS OF INTER TO INTRA SUBJECT VARIANCE

In [18]:
ratios = {}
for condition in conditions:
    condition_dict = {}
    for method in methods:
        ratio = inter_subject_variance[condition][method]/intra_subject_variance[condition][method]
        condition_dict[method] = ratio
    ratios[condition] = condition_dict

In [19]:
ratios

{'EO': {'coh': 171.54485171882348,
  'ciplv': 3.016197567858717,
  'imcoh': 2.3244189368751167,
  'wpli2': 5.7061662735288925},
 'EC': {'coh': 127.42347078182131,
  'ciplv': 13.942390412867256,
  'imcoh': 6.423102903868978,
  'wpli2': 22.874940396142108}}